<a href="https://colab.research.google.com/github/Ab-Rouf/Power-BI-Project/blob/master/Capstone_project_putting_data_science_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Class-6. Capstone Project: Putting Data Science Together

Some Key concepts:
-Data Engineer
-Data Analyst
-Business Analyst
-Business Intelligence

Data Analysing and Visualization tool :power BI, Tableu

"""
import pandas as pd


In [ ]:
products_df=pd.read_csv("products.csv")
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level
0,1001,Organic Rice,Groceries,1551.26,2296.31,43
1,1002,Olive Oil,Groceries,2516.38,3370.60,39
2,1003,Canned Beans,Groceries,961.25,1117.36,30
3,1004,Pasta,Groceries,696.83,1007.94,36
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27


In [ ]:
sales_df=pd.read_csv("/content/sales.csv")
sales_df['sale_date']=pd.to_datetime(sales_df['sale_date']).dt.date
sales_df.head()


,product_id,quantity_sold,sale_date,location,discount
0,1013,4,2024-09-02,Rajshahi,52.178882
1,1064,9,2024-02-25,Sylhet,52.059982
2,1011,2,2024-02-07,Dhaka,7.370982
3,1019,2,2024-09-01,Sylhet,51.219448
4,1011,9,2024-01-21,Chittagong,82.274403


In [ ]:
purchases_df=pd.read_csv("/content/purchases.csv")
purchases_df['purchase_date']=pd.to_datetime(purchases_df['purchase_date']).dt.date
purchases_df.head()


,product_id,quantity_purchased,purchase_date
0,1095,94,2024-08-19
1,1054,23,2024-08-16
2,1058,64,2024-12-18
3,1067,57,2024-06-12
4,1046,91,2024-03-13


# 1. Compute current stock

In [ ]:
product_id=1095

# purchases_df[purchases_df['product_id']==product_id] # show the all data for 1095 product_id in row wise
quantity_purchases=purchases_df[purchases_df['product_id']==product_id]['quantity_purchased'].sum()
quantity_sold=sales_df[sales_df['product_id']==product_id]['quantity_sold'].sum()

current_stock=quantity_purchases - quantity_sold

print(current_stock)



476


In [ ]:
def get_current_stock(product_id):
  quantity_purchased=purchases_df[purchases_df['product_id']==product_id]['quantity_purchased'].sum()
  quantity_sold=sales_df[sales_df['product_id']==product_id]['quantity_sold'].sum()
  current_stock=quantity_purchased - quantity_sold
  return current_stock

print(get_current_stock(1095))

476


In [ ]:
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level
0,1001,Organic Rice,Groceries,1551.26,2296.31,43
1,1002,Olive Oil,Groceries,2516.38,3370.60,39
2,1003,Canned Beans,Groceries,961.25,1117.36,30
3,1004,Pasta,Groceries,696.83,1007.94,36
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27


In [ ]:
#new column add 'current_stock'
products_df['current_stock']=products_df['product_id'].apply(
    lambda product_id: get_current_stock(product_id)
)
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120
3,1004,Pasta,Groceries,696.83,1007.94,36,204
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575


In [ ]:
product=products_df[products_df['product_id']==1095] # 1095 er row er sokol data dekhabe single row
product

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock
94,1095,Gaming Keyboard,Electronics,48301.7,64971.16,40,476


#2. Compute profit per product

In [ ]:
def get_profit(product_id):
  quantity_sold=sales_df[sales_df['product_id']==product_id]['quantity_sold'].sum()
  product=products_df[products_df['product_id']==product_id]
  profit_per_sale=product['selling_price'] -product['cost_price']
  profit_per_sale=profit_per_sale.iloc[0] # oi data frame er zero no. index er value lagbe
  total_profit=quantity_sold * profit_per_sale
  return total_profit

products_df['profit']=products_df['product_id'].apply(
    lambda product_id: get_profit(product_id)
)

products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66


#3. Find the slow moving products

In [ ]:
"""
Slow moving products are thouse which sales < 40 duting 90 days

"""
from datetime import datetime, timedelta
start_date=datetime.strptime('2024-12-31','%Y-%m-%d').date()

cutoff_date=start_date - timedelta(days=90)
print(cutoff_date)

last_90_days_sales=sales_df[
    (sales_df['product_id']==product_id) &
    (sales_df['sale_date']>=cutoff_date)
]

last_90_days_sales
total_recent_sales=last_90_days_sales['quantity_sold'].sum()
print(total_recent_sales)
print(total_recent_sales < 40 )


2024-10-02
76
False


In [ ]:
def slow_moving(product_id):
  start_date=datetime.strptime('2024-12-31','%Y-%m-%d').date()
  cutoff_date=start_date - timedelta(days=90)
  last_90_days_sales=sales_df[
    (sales_df['product_id']==product_id) &
    (sales_df['sale_date']>=cutoff_date)
  ]
  total_recent_sales=last_90_days_sales['quantity_sold'].sum()
  return total_recent_sales < 40

products_df['slow_moving']=products_df['product_id'].apply(
    lambda product_id: slow_moving(product_id)
)

products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True


#4. Flag overstocked and understocked products

In [ ]:
def get_stock_status(product_id):
  product=products_df[products_df['product_id']==product_id].iloc[0]
  stock =product['current_stock']
  reorder=product['reorder_level']
  if stock < reorder : return 'Understocked'
  elif stock > reorder * 15: return 'Overstocked'
  return "Properly Stocked"

products_df['stock_status']=products_df['product_id'].apply(
    lambda product_id: get_stock_status(product_id)
)

products_df.tail(10)

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status
90,1091,Wireless Mouse,Electronics,33026.37,41880.22,47,147,1611400.70,False,Properly Stocked
91,1092,Bluetooth Headphones,Electronics,80300.18,138630.53,20,765,14874239.25,False,Overstocked
92,1093,Smartphone Case,Electronics,26293.27,41963.81,39,213,3745259.06,False,Properly Stocked
93,1094,USB-C Charger,Electronics,71595.00,109763.76,48,423,8816983.56,False,Properly Stocked
94,1095,Gaming Keyboard,Electronics,48301.70,64971.16,40,476,3417239.30,False,Properly Stocked
95,1096,4K Monitor,Electronics,12941.32,22496.29,42,206,2828271.12,False,Properly Stocked
96,1097,Portable SSD,Electronics,78333.86,123756.60,20,336,9357084.44,True,Overstocked
97,1098,Smartwatch,Electronics,32861.41,46319.01,22,153,3216366.40,False,Properly Stocked
98,1099,Fitness Tracker,Electronics,64202.41,111600.88,37,300,11423031.27,False,Properly Stocked
99,1100,Noise Cancelling Earbuds,Electronics,77254.00,128853.90,44,358,11506777.70,False,Properly Stocked


In [ ]:
products_df['stock_status'].value_counts()

,count
stock_status,
Properly Stocked,72
Overstocked,17
Understocked,11


#5. Get Reveneu per product

In [ ]:
def get_revenue(products_df, sales_df, product_id):
  selling_price=products_df[products_df['product_id']
                            ==product_id]['selling_price'].iloc[0]
  quantity_sold=sales_df[sales_df['product_id']==product_id]['quantity_sold'].sum()
  revenue=selling_price * quantity_sold
  return revenue

products_df['revenue']=products_df['product_id'].apply(
    lambda product_id: get_revenue(products_df, sales_df, product_id)
)

products_df.head()


,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status,revenue
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,Overstocked,489114.03
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Properly Stocked,576372.60
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Properly Stocked,239115.04
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Properly Stocked,329596.38
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,Overstocked,484694.28


#6. Queries business data

In [ ]:
def get_sales_between_dates(sales_df,start_date, end_date,locations):
  return sales_df[(sales_df['sale_date']>=start_date) &
                  (sales_df['sale_date']<=end_date) &
                  (sales_df['location'].isin(locations))
                  ]



In [ ]:
products_df['category'].value_counts()

,count
category,
Groceries,30
Electronics,30
Clothing,20
Perishables,20


In [ ]:
def get_products_of_selected_category(products_df, category):
  return products_df[products_df['category'].isin(category)]

In [ ]:
def get_understocked_products(products_df):
  return products_df[products_df['stock_status']=='Understocked']

In [ ]:
def get_summary_kpis(products_df,sales_df):
  total_revenue=products_df['product_id'].apply(
      lambda product_id: get_revenue(products_df, sales_df, product_id)
  ).sum()

  total_profit=products_df['profit'].sum()
  total_units_sold=sales_df['quantity_sold'].sum()
  total_understocked_produtcs=len(get_understocked_products(products_df))
  return {
      'Total Revenue (K)':int(total_revenue/1e3),
      'Total Profit (K)':int(total_profit/1e3),
      'Total Units Sold':int(total_units_sold),
      'Total Understocked Products':(total_understocked_produtcs)
  }


In [ ]:
start_date = datetime.strptime('2024-01-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2024-12-31', '%Y-%m-%d').date()
selected_locations = ['Dhaka']
selected_categories = ['Groceries']


In [ ]:
filtered_sales = get_sales_between_dates(
    sales_df,
    start_date,
    end_date,
    selected_locations
)

filtered_sales.head()

,product_id,quantity_sold,sale_date,location,discount
2,1011,2,2024-02-07,Dhaka,7.370982
5,1077,7,2024-04-19,Dhaka,94.679928
8,1001,6,2024-08-18,Dhaka,56.834725
9,1040,7,2024-01-08,Dhaka,89.031249
11,1076,6,2024-10-16,Dhaka,96.526187


In [ ]:
filtered_products=get_products_of_selected_category(
    products_df,
    selected_categories
)

filtered_products.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status,revenue
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,Overstocked,489114.03
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Properly Stocked,576372.60
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Properly Stocked,239115.04
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Properly Stocked,329596.38
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,Overstocked,484694.28


In [ ]:
understocked_products=get_understocked_products(filtered_products)
understocked_products.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status,revenue
47,1048,Almond Milk,Groceries,2621.61,3401.59,33,-47,180955.36,False,Understocked,789168.88
80,1081,Organic Rice,Groceries,1533.01,2074.05,22,13,153655.36,False,Understocked,589030.20
81,1082,Olive Oil,Groceries,2250.53,2957.89,47,5,200182.88,False,Understocked,837082.87


In [ ]:
get_summary_kpis(filtered_products,filtered_sales)


#visualization using plotly

In [ ]:
import plotly.express as px
top_products = filtered_products.nlargest(10, 'profit')[['product_name', 'profit']]
print(top_products)
plot1 = px.bar(top_products, x='product_name', y='profit', title="Top 10 Products by Profit")
plot1.show()

    product_name     profit
44  Coffee Beans  246773.82
46  Granola Bars  241602.34
81     Olive Oil  200182.88
43         Pasta  197819.76
40  Organic Rice  192514.56
47   Almond Milk  180955.36
0   Organic Rice  158695.65
80  Organic Rice  153655.36
1      Olive Oil  146071.62
48         Honey  142789.50


In [ ]:
category_profit = filtered_products.groupby('category')['profit'].sum().reset_index()
plot2 = px.pie(category_profit, values='profit', names='category', title="Profit Distribution by Category")
plot2.show()

# Tables


In [ ]:
summary_df = filtered_products[['product_name', 'category', 'current_stock', 'reorder_level', 'profit', 'stock_status']]
summary_df.head()

,product_name,category,current_stock,reorder_level,profit,stock_status
0,Organic Rice,Groceries,658,43,158695.65,Overstocked
1,Olive Oil,Groceries,301,39,146071.62,Properly Stocked
2,Canned Beans,Groceries,120,30,33407.54,Properly Stocked
3,Pasta,Groceries,204,36,101732.97,Properly Stocked
4,Coffee Beans,Groceries,575,27,134286.66,Overstocked
